In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv() 
openai_api_key = os.getenv('OPENAI_API_KEY') 


MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
system_message = """You are a helpful assistant for an airline called FlightAI
Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't 
know the answer say so. 
"""

In [5]:
def chat(message, history):
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )

    response = openai.chat.completions.create(model = MODEL, messages=messages) 
    return response.choices[0].message.content 

gr.ChatInterface(fn = chat, type='messages').launch(inline=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city): 
    print(f"Tool get_ticket_price called for {destination_city}") 
    city = destination_city.lower() 
    return ticket_prices.get(city,"Unknown")

In [8]:
get_ticket_price('Berlin')

Tool get_ticket_price called for Berlin


'$499'

In [9]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    },
}

In [11]:
tools= [{'type':'function','function':price_function}]

In [42]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]

    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")

    price = get_ticket_price(city)

    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id,
    }

    return response, city

In [43]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [3]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

In [4]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [34]:
extract_student_info = {
    "name": "extract_student_info",
    "description": "Get the student information from the body of the input text",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Name of person"},
            "major": {"type": "string", "description": "Major subject"},
            "school": {"type": "string", "description": "university name"},
            "grades": {"type": "string", "description": "GPA of the student"},
            "club": {
                "type": "string",
                "description": "School club for extracurricular activities",
            },
        },
        "additionalProperties": False,
    },
}

extract_school_info = {
    "name": "extract_school_info",
    "description": "Get school information from the body of the input text",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Name of the school."},
            "ranking": {
                "type": "integer",
                "description": "QS world ranking of the school.",
            },
            "country": {"type": "string", "description": "Country of the school."},
            "no_of_students": {
                "type": "integer",
                "description": "Number of students enrolled in the school.",
            },
        },
    },
}
tools = [
    {"type": "function", "function": extract_student_info},
    {"type": "function", "function": extract_school_info},
]

In [18]:
student_descriptions = [student_1_description, student_2_description]



In [35]:
for i in student_descriptions: 
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=[{'role':'user', 'content':i}], 
        tools=tools,
        tool_choice='auto'
    )
    tool_calls = response.choices[0].message.tool_calls
    for tool_call in tool_calls: 
        tool_name = tool_call.function.name 
        args = json.loads(tool_call.function.arguments) 
        print(tool_name, args)



extract_student_info {'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': '3.8', 'club': 'Robotics Club'}
extract_school_info {'name': 'Stanford University'}
extract_student_info {'name': 'Ravi Patel', 'major': 'Computer Science', 'school': 'University of Michigan', 'grades': '3.7', 'club': 'Chess Club'}
extract_school_info {'name': 'University of Michigan'}


extract_student_info {'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': '3.8', 'club': 'Robotics Club'}


In [29]:
tool_name

'extract_student_info'

IndexError: list index out of range